<a href="https://colab.research.google.com/github/momchilgeorgiev/2023DL/blob/main/Testing_this_time_it_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Flatten, Conv2D, \
                             Dropout, Input, MaxPool2D, Rescaling, BatchNormalization
from tensorflow.keras.models import Sequential, Model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
(ds_train, ds_test), ds_info = tfds.load(
    'cifar100',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

In [38]:
ds_info.features

FeaturesDict({
    'coarse_label': ClassLabel(shape=(), dtype=int64, num_classes=20),
    'id': Text(shape=(), dtype=string),
    'image': Image(shape=(32, 32, 3), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=100),
})

### Dataset to numpy:

In [5]:
# Create a subset of ds_train and ds_val
ds_train_subset = ds_train.take(500)
ds_val = ds_train.skip(600).take(50)

# Convert datasets to np arrays
def dataset_to_numpy(dataset):
    features = []
    labels = []

    for feature, label in tfds.as_numpy(dataset):
        features.append(feature)
        labels.append(label)

    return np.array(features), np.array(labels)

# Convert ds_train_subset and ds_val to np arrays
np_train_features, np_train_labels = dataset_to_numpy(ds_train_subset)
np_val_features, np_val_labels = dataset_to_numpy(ds_val)

In [6]:
print("-------TRAIN--------")
print(np_train_features.shape)
print(np_train_labels.shape)
print("-------VALIDATION--------")
print(np_val_features.shape)
print(np_val_labels.shape)

-------TRAIN--------
(500, 16, 32, 32, 3)
(500, 16)
-------VALIDATION--------
(50, 16, 32, 32, 3)
(50, 16)


In [7]:
np_test_features, np_test_labels = dataset_to_numpy(ds_test)
print("-------TEST--------")
print(np_test_features.shape)
print(np_test_labels.shape)

-------TEST--------
(625, 16, 32, 32, 3)
(625, 16)


In [8]:
CLASS_NUM = np_test_labels.shape[1]
print(CLASS_NUM)

16


### Preprocessing #2

In [4]:
def preprocess(image, label):
    image = tf.image.per_image_standardization(image)

    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    return image, label

In [9]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64
ds_info.features['label'].num_classes

ds_train = ds_train.cache().shuffle(1000).batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTO)

ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [50]:
tf.keras.backend.clear_session()
model = Sequential([

    Conv2D(32, (3, 3), activation='relu', input_shape=ds_info.features['image'].shape),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.4),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(CLASS_NUM, activation='softmax')

    ])

In [49]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(ds_train, epochs=100, validation_data=ds_test)

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7ed326b6caf0>
Traceback (most recent call last):
  File "/usr/lib/python3.10/weakref.py", line 370, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 


782/782 [==============================] - 84s 10ms/step - loss: 3.2227 - accuracy: 0.2096 - val_loss: 2.8862 - val_accuracy: 0.2846
Epoch 2/100
782/782 [==============================] - 7s 9ms/step - loss: 3.1599 - accuracy: 0.2196 - val_loss: 2.8784 - val_accuracy: 0.2827
Epoch 3/100
782/782 [==============================] - 7s 9ms/step - loss: 3.0925 - accuracy: 0.2324 - val_loss: 2.7200 - val_accuracy: 0.3191
Epoch 4/100
782/782 [==============================] - 7s 9ms/step - loss: 3.0369 - accuracy: 0.2422 - val_loss: 2.6851 - val_accuracy: 0.3264
Epoch 5/100
782/782 [==============================] - 6s 8ms/step - loss: 2.9833 - accuracy: 0.2547 - val_loss: 2.6296 - val_accuracy: 0.3319
Epoch 6/100
782/782 [==============================] - 7s 9ms/step - loss: 2.9267 - accuracy: 0.2640 - val_loss: 2.6121 - val_accuracy: 0.3455
Epoch 7/100
782/782 [==============================] - 6s 8ms/step - loss: 2.8909 - accuracy: 0.2692 - val_loss: 2.5826 - val_accuracy: 0.3430
Epoch 8/1

In [53]:
model.save_weights('model.h5')